In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.shapelets import ShapeletModel, grabocka_params_to_shapelet_size_dict
from tslearn.shapelets import ShapeletModel, grabocka_params_to_shapelet_size_dict
import datetime as dt
import numpy as np
import os
import h5py
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
from feature_engine.encoding import OneHotEncoder
import torchvision.transforms as transforms
from tqdm import tqdm
%matplotlib inline


2024-07-04 22:01:31.253016: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-04 22:01:31.298390: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Specify dataset path to where you downloaded https://www.eurocrops.tum.de/downloads.html
dataset_dir = "/home/fin_eckhoff/Projects/TimeSeries/TimeSeries"

# Get all paths
train_h5_dir  = os.path.join(dataset_dir, 'm1615987/HDF5s/train/')
train_csv_dir = os.path.join(dataset_dir, 'm1615987/csv_labels/train/')
test_h5_dir   = os.path.join(dataset_dir, 'm1615987/HDF5s/test/')
test_csv_dir  = os.path.join(dataset_dir, 'm1615987/csv_labels/test/')

# check they all exist
print(os.path.exists(test_csv_dir))
print(os.path.exists(test_h5_dir))
print(os.path.exists(train_csv_dir))
print(os.path.exists(train_h5_dir))


True
True
True
True


In [3]:
selected_file = os.path.join(train_h5_dir, 'AT_T33UWP_train.h5')

# load the .h5 file using pandas.
hdf = pd.HDFStore(selected_file, mode='r') 

# list all the keys, which are regions, in the country-region?
print(hdf.keys())

# extract a subset - the first region from the list
df_data1 = hdf.get(hdf.keys()[0])

['/AT112', '/AT122', '/AT123', '/AT124', '/AT125', '/AT127', '/AT130', '/AT223']


In [4]:
h5_file = h5py.File(selected_file, 'r')
regions =  list(h5_file.keys())

In [5]:
csv_file_path = os.path.join(train_csv_dir, 'demo_eurocrops_' + regions[0] + '.csv')
print(csv_file_path)
print("Exists:", os.path.exists(csv_file_path))

df_labels = pd.read_csv(csv_file_path, index_col=0)

/home/fin_eckhoff/Projects/TimeSeries/TimeSeries/m1615987/csv_labels/train/demo_eurocrops_AT112.csv
Exists: True


In [6]:
labels = df_labels[["crpgrpn"]].to_numpy()
encoder = OneHotEncoder()
one_hot_encoded_labels = encoder.fit_transform(labels)

In [7]:
recno = df_labels["crpgrpn"].index.to_list()

In [33]:
one_hot_encoded_labels

,x0_leguminous_plants,x0_fruit_of_temperate_climate_zones,x0_other_cereals_for_the_production_of_grain,x0_pasture_meadow,x0_nuts,x0_summer_common_wheat_and_spelt,x0_winter_common_wheat_and_spelt,x0_fresh_vegetables_melons_and_strawberries,x0_other_plants_harvested_green,x0_aromatic_plants_medicinal_and_culinary_plants,...,x0_sunflower_and_yellow_bloomer,x0_potatoes,x0_vineyards,x0_not_known,x0_winter_barley,x0_winter_durum_wheat,x0_winter_rape,x0_winter_rye,x0_winter_triticale,x0_sugar_beet
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6259,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
df_labels["crpgrpn"].value_counts()


crpgrpn
winter_common_wheat_and_spelt                     1381
pasture_meadow                                     985
grain_maize                                        766
soya                                               600
other_plants_harvested_green                       395
vineyards                                          366
winter_barley                                      244
sunflower_and_yellow_bloomer                       244
others                                             215
leguminous_plants                                  156
sugar_beet                                         151
winter_triticale                                   113
fresh_vegetables_melons_and_strawberries           102
winter_rye                                          74
winter_rape                                         67
millet                                              66
winter_durum_wheat                                  65
summer_durum_wheat                                  64
su

In [9]:
class EurocropDataset(Dataset):
    
  
    
    def __init__(self, time_series_data, labels, mean = None, std = None):
        self.time_series_data = time_series_data
        self.labels = labels
        
        self.min = torch.tensor(self.time_series_data.min()[0])
        self.max = torch.tensor(self.time_series_data.max()[0])
        
        print(self.min, self.max)
        #self.data = 2 * (self.time_series_data - self.min) / (self.max - self.min) - 1
        
        
    def __len__(self):
        return len(self.time_series_data)
    def __getitem__(self, idx):
        series = self.time_series_data.iloc[idx]
        label = self.labels.iloc[idx]
        series_tensor = torch.tensor(series, dtype=torch.float32)
        series_tensor = 2 * (series_tensor - self.min) / (self.max - self.min) - 1
        label_tensor = torch.tensor(label, dtype=torch.float32 )
        return series_tensor, label_tensor

In [10]:
ds = EurocropDataset(df_data1, one_hot_encoded_labels)

/tmp/ipykernel_1699120/1879154529.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.min = torch.tensor(self.time_series_data.min()[0])


tensor([1757, 1286,  820,  615,  618,  583,  595,  545,  288,   15,  444,  360,
         580]) tensor([ 7456,  7986,  7603,  8994,  9521,  9752,  9961,  9872,  6116,   262,
         8295,  6585, 10354])


/tmp/ipykernel_1699120/1879154529.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.max = torch.tensor(self.time_series_data.max()[0])


In [11]:
next(iter(ds))[0].shape

/tmp/ipykernel_1699120/1879154529.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  series_tensor = torch.tensor(series, dtype=torch.float32)
/tmp/ipykernel_1699120/1879154529.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label_tensor = torch.tensor(label, dtype=torch.float32 )


torch.Size([80, 13])

In [12]:
dl = DataLoader(ds, batch_size=512, shuffle=True)

________________________________________________________________

In [13]:
import tensorflow.keras.backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Bidirectional, Dropout
from keras.utils import to_categorical
from keras.regularizers import l2
from keras import Model


In [15]:
def generate_model():
    recurrent_input = Input(shape=(80,13),name="TIMESERIES_INPUT")
    print(recurrent_input.shape)
    
    rec_layer_one = Bidirectional(LSTM(128, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01),return_sequences=True),name ="BIDIRECTIONAL_LAYER_1")(recurrent_input)
    rec_layer_one = Dropout(0.1,name ="DROPOUT_LAYER_1")(rec_layer_one)
    print(rec_layer_one.shape)
    
    rec_layer_two = Bidirectional(LSTM(64, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)),name ="BIDIRECTIONAL_LAYER_2")(rec_layer_one)
    rec_layer_two = Dropout(0.1,name ="DROPOUT_LAYER_2")(rec_layer_two)
    
    print(rec_layer_two.shape)
    
    
    output = Dense(31,activation='softmax',name="OUTPUT_LAYER")(rec_layer_two)
    print(output.shape)
    model = Model(inputs=[recurrent_input],outputs=[output])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1_m,precision_m, recall_m])
    return model

In [16]:
rnn_model = generate_model()

2024-07-04 22:01:41.191228: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-04 22:01:41.191466: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-04 22:01:41.192779: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

(None, 80, 13)


L355
2024-07-04 22:01:41.374701: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-04 22:01:41.374858: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-04 22:01:41.409144: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#

(None, 80, 256)
(None, 128)
(None, 31)


L344-L355
2024-07-04 22:01:41.409407: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-04 22:01:41.409602: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-04 22:01:41.409765: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df_data1, one_hot_encoded_labels, test_size=0.2, random_state=42)

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [19]:
checkpoint_callback = ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    mode='max',
    verbose=1
)

In [20]:
rnn_model.fit(dl, epochs=200, callbacks=[checkpoint_callback])

/tmp/ipykernel_1699120/1879154529.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  series_tensor = torch.tensor(series, dtype=torch.float32)
/tmp/ipykernel_1699120/1879154529.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label_tensor = torch.tensor(label, dtype=torch.float32 )


Epoch 1/200


2024-07-04 22:01:46.819877: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 159ms/step - accuracy: 0.0449 - f1_m: 0.0000e+00 - loss: 9.2507 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
Epoch 2/200


/home/fin_eckhoff/miniforge-pypy3/envs/ML/lib/python3.12/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 171ms/step - accuracy: 0.0920 - f1_m: 0.0000e+00 - loss: 6.1601 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 167ms/step - accuracy: 0.1519 - f1_m: 0.0000e+00 - loss: 4.0348 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 168ms/step - accuracy: 0.2227 - f1_m: 0.0000e+00 - loss: 2.6379 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 168ms/step - accuracy: 0.2166 - f1_m: 0.0000e+00 - loss: 1.7305 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.2197 - f1_m: 0.0000e+00 - loss: 1.1454 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 167ms/step - accuracy: 0.2254 - f1_m: 0.0000e+00 - loss: 0.7716 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 168ms/step - accuracy: 0.2099 - f1_m: 0.

In [21]:
x_test, y_test = next(iter(dl))

/tmp/ipykernel_1699120/1879154529.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  series_tensor = torch.tensor(series, dtype=torch.float32)
/tmp/ipykernel_1699120/1879154529.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label_tensor = torch.tensor(label, dtype=torch.float32 )


In [22]:
x_test.shape

torch.Size([512, 80, 13])

In [29]:
id= 55
x = x_test[id].reshape((1,80,13))
y_pred = rnn_model.predict(x)
predicted_classes = np.argmax(y_pred, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


In [24]:
x

tensor([[[ 0.0777,  0.0451, -0.0458,  ..., -0.1756, -0.1833, -0.1283],
         [-0.3978, -0.5087, -0.4990,  ..., -0.7881, -0.8043, -0.1414],
         [-1.0256, -0.9881, -0.9307,  ..., -0.5043, -0.6514, -0.5582],
         ...,
         [-0.5978, -0.7179, -0.6984,  ..., -0.6607, -0.6768, -0.6354],
         [-0.9912, -0.9875, -0.9717,  ..., -0.8657, -0.9406, -0.8819],
         [-0.1205, -0.1504, -0.2360,  ..., -0.2075, -0.1470, -0.2509]]])

In [25]:
y_test[id]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [30]:
print(predicted_classes)
print(np.argmax(y_test[id], axis=0))

[6]
tensor(21)
